# Set up

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm
from linear_operator import settings

import pyro
import math
import pickle
import time
from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import arviz as az
import seaborn as sns

In [2]:
import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools

In [3]:
Device = 'cpu'

# Read data

In [4]:
TrainData = pd.read_csv('Data/train3D.csv', delimiter=',')
TestData = pd.read_csv('Data/test3D.csv', delimiter=',')

scaler = StandardScaler()
TrainData_standardized = pd.DataFrame(scaler.fit_transform(TrainData), columns=TrainData.columns).values

TestData_standardized = pd.DataFrame(scaler.fit_transform(TestData), columns=TestData.columns).values

In [5]:
X_train = TrainData_standardized[:,0]
Y_train = TrainData_standardized[:,-1]

X_test = TestData_standardized[:,0]
Y_test = TestData_standardized[:,-1]

In [6]:
train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

train_y = torch.tensor(Y_train, dtype=torch.float32)
test_y = torch.tensor(Y_test, dtype=torch.float32)

# Training model

In [ ]:
LocalGP_models, LocalGP_likelihoods = Training.train_one_column_StandardGP(
    train_x, train_y, covar_type = 'RBF', lr=0.05, num_iterations=5000, patience=10, device='cpu'
    )

# Prediction

In [8]:
preds = Prediction.preds_for_one_model(LocalGP_models, LocalGP_likelihoods, test_x.to(Device)).cpu().detach().numpy()

In [ ]:
np.mean((preds - test_y.numpy()) ** 2)

# Estimation

## Point estimation

In [ ]:
row_idx = 0

bounds = bound.get_bounds(train_x)

estimated_params, func_loss = Estimation.multi_start_estimation(LocalGP_models, LocalGP_likelihoods, row_idx, test_y, bounds, Estimation.estimate_params_for_one_model_Adam, 
                                                                num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                attraction_threshold=0.1, repulsion_strength=0.1, device=Device)

In [ ]:
test_x[row_idx]

In [ ]:
estimated_params

## MCMC 

In [ ]:
mcmc_result = Estimation.run_mcmc(Prediction.preds_for_one_model, LocalGP_models, LocalGP_likelihoods, row_idx, test_y, bounds, 
                                                  num_sampling = 400, warmup_step = 100)



In [ ]:
posterior_samples_Normal = mcmc_result.get_samples()

true_values = test_x[row_idx,:]

point_estimations = 'None'

fig, axes = plt.subplots(len(posterior_samples_Normal), 1, figsize=(8, len(posterior_samples_Normal) * 3))

for i, (param_name, samples) in enumerate(posterior_samples_Normal.items()):
    if len(posterior_samples_Normal) > 1:
        ax = axes[i]
    else:
        ax = axes
    sns.kdeplot(samples.detach().numpy(), ax=ax, color='blue')
    ax.set_title(f'Density of {param_name}')
    
    if true_values is not None and i < len(true_values):
        ax.axvline(true_values[i], color='red', linestyle='--', label='True Value')
    if point_estimations is not None and i < len(point_estimations):
        ax.axvline(point_estimations[i], color='green', linestyle='-.', label='Point Estimation')
    ax.legend()

plt.tight_layout()
plt.show()